## SQL Query

###### To filter rows by specific criteria, we need to use the WHERE statement. The WHERE statement requires three things:

* The column we want the database to filter on: ShareWomen
* A comparison operator that specifies how we want to compare a value in a column: >
* The value we want the database to compare each value to: 0.5 

###### LIMIT - Limiting the Number of Results
> SELECT Major FROM recent_grads LIMIT 5;

###### SQL's comparison operators


* Less than: <
* Less than or equal to: <=
* Greater than: >
* Greater than or equal to: >=
* Equal to: =
* Not equal to: !=

###### Logical operators


* __OR__ (returns either Condition1 or Condition2)
* __AND__ (returns both Condition1 and Condition2)

###### Grouping Operators With Parentheses

> select Major, Major_category, ShareWomen, Unemployment_rate  
> from recent_grads  
> where (Major_category = 'Engineering') and (ShareWomen > 0.5 or Unemployment_rate < 0.051);


###### Connecting to the database
> __connect()__  

__Cursor Class__ is used for:

> Run a query against the database  
> Parse the results from the database  
> Convert the results to native Python objects  
> Store the results within the Cursor instance as a local variable  

* A __tuple__ is a core data structure that Python uses to represent a sequence of values, similar to a list. Unlike lists, tuples are immutable, which means we can't modify existing ones.  

Creating an empty tuple:
> t = ()

* Tuples are faster than lists, so they're helpful with larger databases and larger results sets. 

* Connection instance method __cursor()__ is used to return a Cursor instance corresponding to the database we want to query

> cursor = conn.__cursor()__

* SQLite library actually allows us to skip creating a Cursor altogether by using the __execute__ method within the __Connection__ object itself. SQLite will create a __Cursor__ instance for us under the hood and our query run against the database, but this shortcut allows us to skip a step

> conn.__execute__(query).fetchall()

* Fetching Results - 

> __fetchall__ -- returns all results  
> __fetchmany(n)__ -- returns n results specified by the parameter n  
> __fetchone__ -- returns one result at a time, behind the hood cursor will keep track of fetched result and will give us  a new row each time we call the function  

* closing connection

> important to close connection as open connections restrits access to the database  
> SQLite will save any changes made into the database during this time  

In [2]:
import sqlite3
conn = sqlite3.connect('test.db')

In [1]:
import pandas as pd
recent_grads = pd.read_csv('/media/mostafa/Study/DQ/data/college-majors/recent-grads.csv')
recent_grads[:1]


,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,Employed,...,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,1976,...,270,1207,37,0.018381,110000,95000,125000,1534,364,193


In [3]:
cursor = conn.cursor()
recent_grads.to_sql('recent_grads',conn)

In [15]:
query = 'select distinct rank,major from recent_grads;'
cursor.execute(query)
result = cursor.fetchall()
result[:5]

[(1, 'PETROLEUM ENGINEERING'),
 (2, 'MINING AND MINERAL ENGINEERING'),
 (3, 'METALLURGICAL ENGINEERING'),
 (4, 'NAVAL ARCHITECTURE AND MARINE ENGINEERING'),
 (5, 'CHEMICAL ENGINEERING')]

In [17]:
query = 'select distinct rank,major from recent_grads;'
result = conn.execute(query).fetchall()
result[:5]

[(1, 'PETROLEUM ENGINEERING'),
 (2, 'MINING AND MINERAL ENGINEERING'),
 (3, 'METALLURGICAL ENGINEERING'),
 (4, 'NAVAL ARCHITECTURE AND MARINE ENGINEERING'),
 (5, 'CHEMICAL ENGINEERING')]

In [3]:
conn.close()

In [19]:
conn.execute('select * from recent_grads limit 2;')

ProgrammingError: Cannot operate on a closed database.

In [2]:
import sqlite3
conn = sqlite3.connect('factbook.db')
query = 'select * from facts limit 1;'
facts = conn.execute(query).fetchall()
facts

[(1,
  'af',
  'Afghanistan',
  652230,
  652230,
  0,
  32564342,
  2.32,
  38.57,
  13.89,
  1.51,
  '2015-11-01 13:19:49.461734',
  '2015-11-01 13:19:49.461734')]

In [10]:
import math
import pandas as pd
# reading data into pandas datafram
query = 'select * from facts;'
facts = pd.read_sql_query(query,conn)
facts.shape

# filtering out data,dropping nulls on the column area land
facts = facts.dropna(subset=['area_land','population','population_growth'])
# filtering out data, discarding 0 on area land
facts = facts[(facts['area_land'] != 0) & (facts['population'] != 0) & (facts['population_growth'] != 0)]

pop = facts['population'][:1]
growth = facts['population_growth'][:1]
print(pop*math.pow(growth*35,math.e))

facts[:1]

0    5.051889e+12
Name: population, dtype: float64


,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51,2015-11-01 13:19:49.461734,2015-11-01 13:19:49.461734


In [9]:
print(math.e*pop)

0    8.851906e+07
Name: population, dtype: float64


In [1]:
import math
import pandas as pd
# reading data into pandas datafram
query = 'select * from facts;'
facts = pd.read_sql_query(query,conn)
facts.shape

# filtering out data,dropping nulls on the column area land
facts = facts.dropna(subset=['area_land','population','population_growth'])
# filtering out data, discarding 0 on area land
facts = facts[(facts['area_land'] != 0) & (facts['population'] != 0) & (facts['population_growth'] != 0)]
facts.shape

#project population of future year
def project_population(initial_population, population_growth):
    return  int(initial_population * math.pow(population_growth * 35,math.e))

facts['population_2050'] = facts.apply(lambda x: project_population(x['population'],x['population_growth']),axis=1)
facts = facts.sort_values('population_2050',ascending=0)
print(facts[:10])


NameError: name 'conn' is not defined

In [16]:
query = 'select count(birth_rate) from facts;'
birth_rate_count = conn.execute(query).fetchall()[0][0]
birth_rate_count


228

In [13]:
len(facts)

1